In [ ]:
import os
import sys
import time
import logging
import argparse
import numpy as np
import pandas as pd
from tqdm import tqdm
from random import random
import itertools
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns

import torch

選好

In [1]:
def make_pref(n,m):
    row = torch.linspace(1/m, 1, m)  # 1行分の値を生成
    pref = torch.stack([row[torch.randperm(m)] for _ in range(n)])
    return pref

サイクル１つ見つけたらそこで終了してしまう

In [2]:
class RandomAssignmentAdjuster:
    def __init__(self, P, preferences):
        """
        P: n x n の二重確率行列 (torch.Tensor)
        preferences: n x n の選好行列 (torch.Tensor)
                     各行 i はエージェント i の選好を表し、値が大きいほど好む
        """
        self.P = P
        self.preferences = preferences
        self.n = P.shape[0]

    def build_graph(self):
        """
        オブジェクトを頂点とするグラフを構築する。
        エッジ (a -> b) は、あるエージェント i が a を b より好む（preferences[i, a] > preferences[i, b]）
        かつ P[i, b] > 0 である場合に追加する。
        エッジには (b, i, P[i,b]) の情報を記録する。
        """
        graph = {a: [] for a in range(self.n)}
        for a in range(self.n):
            for b in range(self.n):
                if a == b:
                    continue
                for i in range(self.n):
                    if self.preferences[i, a] > self.preferences[i, b] and self.P[i, b] > 0:
                        graph[a].append((b, i, self.P[i, b].item()))
                        # 同じ (a, b) ペアに対しては最初に見つかったエージェントのみ利用
                        break
        return graph

    def find_cycle(self, graph):
        """
        DFS を用いてグラフ中のサイクル（閉路）を探索する。
        サイクルが見つかった場合は、サイクルを構成するエッジのリストを返す。
        各エッジは (from_object, to_object, witness_agent, available_probability) のタプル。
        サイクルがなければ None を返す。
        """
        visited = set()
        rec_stack = []  # 各要素は (vertex, edge_info)。最初の頂点は edge_info=None

        def dfs(v):
            visited.add(v)
            rec_stack.append((v, None))
            for (nbr, agent, avail) in graph[v]:
                # すでに経路に nbr が含まれていればサイクル成立
                for idx, (node, _) in enumerate(rec_stack):
                    if node == nbr:
                        cycle_edges = []
                        # rec_stack[idx+1:] に記録されている各エッジ情報がサイクル内のエッジ
                        for j in range(idx + 1, len(rec_stack)):
                            edge = rec_stack[j][1]
                            if edge is not None:
                                cycle_edges.append(edge)
                        # 現在のエッジ (v -> nbr) を追加
                        cycle_edges.append((v, nbr, agent, avail))
                        return cycle_edges
                if nbr not in visited:
                    # 現在のエッジ (v -> nbr) を経路に記録して DFS
                    rec_stack.append((v, (v, nbr, agent, avail)))
                    result = dfs(nbr)
                    if result is not None:
                        return result
                    rec_stack.pop()
            rec_stack.pop()
            return None

        for vertex in range(self.n):
            if vertex not in visited:
                cycle = dfs(vertex)
                if cycle is not None:
                    return cycle
        return None

    def execute_cycle(self):
        """
        グラフを構築し、サイクルを探索する。
        サイクルが存在する場合、サイクル内の各エッジで交換可能な最小の確率 epsilon
        を算出し、その分だけ、サイクル内の各エッジ (a -> b) に対して、証人エージェント i の行
        で、オブジェクト a の割当を epsilon 増加、オブジェクト b の割当を epsilon 減少する。
        更新後の二重確率行列 Q、epsilon、サイクル内のエッジ情報を返す。
        サイクルが存在しない場合は P をそのまま返す。
        """
        graph = self.build_graph()
        cycle = self.find_cycle(graph)
        if cycle is None:
            print("サイクルが存在しません。P は順序効率的です。")
            return self.P, 0.0, None
        
        # サイクル内の各エッジの available_probability の最小値 epsilon を求める
        epsilons = [edge[3] for edge in cycle]
        epsilon = min(epsilons)
        
        # P のコピー Q を作成して更新
        Q = self.P.clone()
        for (a, b, agent, avail) in cycle:
            Q[agent, b] -= epsilon
            Q[agent, a] += epsilon
        return Q, epsilon, cycle



In [3]:
if __name__ == "__main__":
    # 例: 4エージェント・4オブジェクトの場合
    P = torch.tensor([
        [0.5, 0.25, 0.25, 0.0],
        [0.25, 0.5, 0.25, 0.0],
        [0.25, 0.25, 0.5, 0.0],
        [0.0, 0.0, 0.0, 1.0]
    ])
    
    preferences = make_pref(4, 4)
    
    adjuster = RandomAssignmentAdjuster(P, preferences)
    
    print("元の二重確率行列 P:")
    print(P)
    print("\n選好行列:")
    print(preferences)
    
    Q, exchanged, cycle = adjuster.execute_cycle()
    
    if cycle is not None:
        print("\n検出されたサイクル (各タプルは (from_object, to_object, witness_agent, available_probability)):")
        for edge in cycle:
            print(edge)
        print("\nサイクル内で交換された確率 (epsilon):", exchanged)
        print("\n更新後の二重確率行列 Q:")
        print(Q)
    else:
        print("\nP はすでに順序効率的です。")

NameError: name 'torch' is not defined

In [ ]:

# 例: 4エージェント・4オブジェクトの場合
P = torch.tensor([
    [0.5, 0.25, 0.25, 0.0],
    [0.25, 0.5, 0.25, 0.0],
    [0.25, 0.25, 0.5, 0.0],
    [0.0, 0.0, 0.0, 1.0]
])
    
preferences = make_pref(4, 4)

adjuster = RandomAssignmentAdjuster(P, preferences)

print("元の二重確率行列 P:")
print(P)
print("\n選好行列:")
print(preferences)

Q, exchanged, cycle = adjuster.execute_cycle()

if cycle is not None:
    print("\n検出されたサイクル (各タプルは (from_object, to_object, witness_agent, available_probability)):")
    for edge in cycle:
        print(edge)
    print("\nサイクル内で交換された確率 (epsilon):", exchanged)
    print("\n更新後の二重確率行列 Q:")
    print(Q)
else:
    print("\nP はすでに順序効率的です。")

元の二重確率行列 P:
tensor([[0.5000, 0.2500, 0.2500, 0.0000],
        [0.2500, 0.5000, 0.2500, 0.0000],
        [0.2500, 0.2500, 0.5000, 0.0000],
        [0.0000, 0.0000, 0.0000, 1.0000]])

選好行列:
tensor([[0.7500, 1.0000, 0.5000, 0.2500],
        [0.5000, 0.7500, 1.0000, 0.2500],
        [0.2500, 0.7500, 1.0000, 0.5000],
        [0.5000, 1.0000, 0.2500, 0.7500]])

検出されたサイクル (各タプルは (from_object, to_object, witness_agent, available_probability)):
(0, 2, 0, 0.25)
(2, 0, 1, 0.25)

サイクル内で交換された確率 (epsilon): 0.25

更新後の二重確率行列 Q:
tensor([[0.7500, 0.2500, 0.0000, 0.0000],
        [0.0000, 0.5000, 0.5000, 0.0000],
        [0.2500, 0.2500, 0.5000, 0.0000],
        [0.0000, 0.0000, 0.0000, 1.0000]])


３人も対応している

In [14]:
P = torch.tensor([
        [0.4, 0.3, 0.0, 0.3],
        [0.0, 0.4, 0.3, 0.3],
        [0.3, 0.3, 0.4, 0.0],
        [0.3, 0.0, 0.3, 0.4]
    ])

preferences = torch.tensor([
        [1.00, 0.75, 0.5,  0.25],   # Agent0
        [0.75, 1.00, 0.5,  0.25],   # Agent1
        [0.75, 0.5,  1.00, 0.25],   # Agent2
        [0.75, 0.5,  0.25, 1.00]     # Agent3
    ])

adjuster = RandomAssignmentAdjuster(P, preferences)

print("元の二重確率行列 P:")
print(P)
print("\n選好行列:")
print(preferences)

Q, exchanged, cycle = adjuster.execute_cycle()

if cycle is not None:
    print("\n検出されたサイクル (各タプルは (from_object, to_object, witness_agent, available_probability)):")
    for edge in cycle:
        print(edge)
    print("\nサイクル内で交換された確率 (epsilon):", exchanged)
    print("\n更新後の二重確率行列 Q:")
    print(Q)
else:
    print("\nP はすでに順序効率的です。")

元の二重確率行列 P:
tensor([[0.4000, 0.3000, 0.0000, 0.3000],
        [0.0000, 0.4000, 0.3000, 0.3000],
        [0.3000, 0.3000, 0.4000, 0.0000],
        [0.3000, 0.0000, 0.3000, 0.4000]])

選好行列:
tensor([[1.0000, 0.7500, 0.5000, 0.2500],
        [0.7500, 1.0000, 0.5000, 0.2500],
        [0.7500, 0.5000, 1.0000, 0.2500],
        [0.7500, 0.5000, 0.2500, 1.0000]])


AttributeError: 'RandomAssignmentAdjuster' object has no attribute 'execute_cycle'

In [ ]:
P = torch.tensor([
        [0.4, 0.3, 0.2, 0.1],
        [0.1, 0.4, 0.3, 0.2],
        [0.2, 0.1, 0.4, 0.3],
        [0.3, 0.2, 0.1, 0.4]
    ])

preferences = torch.tensor([
        [1.00, 0.75, 0.5,  0.25],   # エージェント0
        [0.75, 1.00, 0.5,  0.25],   # エージェント1
        [0.5,  0.25, 1.00, 0.75],   # エージェント2
        [0.25, 0.5,  0.75, 1.00]     # エージェント3
    ])

adjuster = RandomAssignmentAdjuster(P, preferences)

print("元の二重確率行列 P:")
print(P)
print("\n選好行列:")
print(preferences)

Q, exchanged, cycle = adjuster.execute_cycle()

if cycle is not None:
    print("\n検出されたサイクル (各タプルは (from_object, to_object, witness_agent, available_probability)):")
    for edge in cycle:
        print(edge)
    print("\nサイクル内で交換された確率 (epsilon):", exchanged)
    print("\n更新後の二重確率行列 Q:")
    print(Q)
else:
    print("\nP はすでに順序効率的です。")

元の二重確率行列 P:
tensor([[0.4000, 0.3000, 0.2000, 0.1000],
        [0.1000, 0.4000, 0.3000, 0.2000],
        [0.2000, 0.1000, 0.4000, 0.3000],
        [0.3000, 0.2000, 0.1000, 0.4000]])

選好行列:
tensor([[1.0000, 0.7500, 0.5000, 0.2500],
        [0.7500, 1.0000, 0.5000, 0.2500],
        [0.5000, 0.2500, 1.0000, 0.7500],
        [0.2500, 0.5000, 0.7500, 1.0000]])

検出されたサイクル (各タプルは (from_object, to_object, witness_agent, available_probability)):
(0, 1, 0, 0.30000001192092896)
(1, 0, 1, 0.10000000149011612)

サイクル内で交換された確率 (epsilon): 0.10000000149011612

更新後の二重確率行列 Q:
tensor([[0.5000, 0.2000, 0.2000, 0.1000],
        [0.0000, 0.5000, 0.3000, 0.2000],
        [0.2000, 0.1000, 0.4000, 0.3000],
        [0.3000, 0.2000, 0.1000, 0.4000]])


In [ ]:
P = torch.tensor([
        [0.7, 0.3, 0.0, 0.0],  # Agent0
        [0.1, 0.4, 0.5, 0.0],  # Agent1
        [0.0, 0.2, 0.4, 0.4],  # Agent2
        [0.2, 0.1, 0.1, 0.6]   # Agent3
    ])

preferences = torch.tensor([
        [1.00, 0.75, 0.5,  0.25],  # Agent0: object0 > object1 > object2 > object3
        [0.50, 1.00, 0.75, 0.25],   # Agent1: object1 > object2 > object0 > object3
        [0.50, 0.25, 1.00, 0.75],   # Agent2: object2 > object3 > object0 > object1  ※edge (object2→object3)用
        [0.25, 0.50, 0.75, 1.00]    # Agent3: object3 > object0 > object1 > object2  ※edge (object3→object0)用
    ])

adjuster = RandomAssignmentAdjuster(P, preferences)

print("元の二重確率行列 P:")
print(P)
print("\n選好行列:")
print(preferences)

Q, exchanged, cycle = adjuster.execute_cycle()

if cycle is not None:
    print("\n検出されたサイクル (各タプルは (from_object, to_object, witness_agent, available_probability)):")
    for edge in cycle:
        print(edge)
    print("\nサイクル内で交換された確率 (epsilon):", exchanged)
    print("\n更新後の二重確率行列 Q:")
    print(Q)
else:
    print("\nP はすでに順序効率的です。")

元の二重確率行列 P:
tensor([[0.7000, 0.3000, 0.0000, 0.0000],
        [0.1000, 0.4000, 0.5000, 0.0000],
        [0.0000, 0.2000, 0.4000, 0.4000],
        [0.2000, 0.1000, 0.1000, 0.6000]])

選好行列:
tensor([[1.0000, 0.7500, 0.5000, 0.2500],
        [0.5000, 1.0000, 0.7500, 0.2500],
        [0.5000, 0.2500, 1.0000, 0.7500],
        [0.2500, 0.5000, 0.7500, 1.0000]])

検出されたサイクル (各タプルは (from_object, to_object, witness_agent, available_probability)):
(0, 1, 0, 0.30000001192092896)
(1, 0, 1, 0.10000000149011612)

サイクル内で交換された確率 (epsilon): 0.10000000149011612

更新後の二重確率行列 Q:
tensor([[0.8000, 0.2000, 0.0000, 0.0000],
        [0.0000, 0.5000, 0.5000, 0.0000],
        [0.0000, 0.2000, 0.4000, 0.4000],
        [0.2000, 0.1000, 0.1000, 0.6000]])


In [ ]:
P = torch.tensor([
        [0.7, 0.3, 0.0, 0.0],  # Agent0
        [0.3, 0.7, 0.0, 0.0],  # Agent1
        [0.0, 0.0, 0.6, 0.4],  # Agent2
        [0.0, 0.0, 0.4, 0.6]   # Agent3
    ])

preferences = torch.tensor([
        [1.00, 0.75, 0.5,  0.25], 
        [0.75, 1.00, 0.5, 0.25], 
        [0.50, 0.25, 1.00, 0.75], 
        [0.25, 0.50, 0.75, 1.00]
    ])

adjuster = RandomAssignmentAdjuster(P, preferences)

print("元の二重確率行列 P:")
print(P)
print("\n選好行列:")
print(preferences)

Q, exchanged, cycle = adjuster.execute_cycle()

if cycle is not None:
    print("\n検出されたサイクル (各タプルは (from_object, to_object, witness_agent, available_probability)):")
    for edge in cycle:
        print(edge)
    print("\nサイクル内で交換された確率 (epsilon):", exchanged)
    print("\n更新後の二重確率行列 Q:")
    print(Q)
else:
    print("\nP はすでに順序効率的です。")

元の二重確率行列 P:
tensor([[0.7000, 0.3000, 0.0000, 0.0000],
        [0.3000, 0.7000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.6000, 0.4000],
        [0.0000, 0.0000, 0.4000, 0.6000]])

選好行列:
tensor([[1.0000, 0.7500, 0.5000, 0.2500],
        [0.7500, 1.0000, 0.5000, 0.2500],
        [0.5000, 0.2500, 1.0000, 0.7500],
        [0.2500, 0.5000, 0.7500, 1.0000]])

検出されたサイクル (各タプルは (from_object, to_object, witness_agent, available_probability)):
(0, 1, 0, 0.30000001192092896)
(1, 0, 1, 0.30000001192092896)

サイクル内で交換された確率 (epsilon): 0.30000001192092896

更新後の二重確率行列 Q:
tensor([[1.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 1.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.6000, 0.4000],
        [0.0000, 0.0000, 0.4000, 0.6000]])


複数サイクル

In [ ]:
import torch

class RandomAssignmentAdjuster:
    def __init__(self, P, preferences):
        """
        P: n×n の二重確率行列 (torch.Tensor)
        preferences: n×n の選好行列 (torch.Tensor)
                     各行 i はエージェント i の選好を表し、値が大きいほど好む
        """
        self.P = P.clone()
        self.preferences = preferences
        self.n = P.shape[0]

    def build_graph(self, Q):
        """
        Q: 現在の二重確率行列 (torch.Tensor)
        オブジェクトを頂点とするグラフを構築する。
        エッジ (a -> b) は、あるエージェント i が a を b より好む（preferences[i, a] > preferences[i, b]）
        かつ Q[i, b] > 0 である場合に追加する。
        エッジには (b, i, Q[i, b]) の情報を記録する。
        """
        graph = {a: [] for a in range(self.n)}
        for a in range(self.n):
            for b in range(self.n):
                if a == b:
                    continue
                for i in range(self.n):
                    if self.preferences[i, a] > self.preferences[i, b] and Q[i, b] > 0:
                        graph[a].append((b, i, Q[i, b].item()))
                        # 同じ (a, b) ペアについて、最初に条件を満たしたエージェントを witness とする
                        break
        return graph

    def find_cycle(self, graph):
        """
        DFS を用いてグラフ中のサイクル（閉路）を探索する。
        サイクルが見つかった場合は、サイクルを構成するエッジのリストを返す。
        各エッジは (from_object, to_object, witness_agent, available_probability) のタプル。
        サイクルがなければ None を返す。
        """
        visited = set()
        rec_stack = []  # 各要素は (vertex, edge_info)。最初の頂点は edge_info=None

        def dfs(v):
            visited.add(v)
            rec_stack.append((v, None))
            for (nbr, agent, avail) in graph[v]:
                for idx, (node, _) in enumerate(rec_stack):
                    if node == nbr:
                        cycle_edges = []
                        # rec_stack[idx+1:] に記録されているエッジ情報がサイクル内のエッジ
                        for j in range(idx + 1, len(rec_stack)):
                            edge = rec_stack[j][1]
                            if edge is not None:
                                cycle_edges.append(edge)
                        cycle_edges.append((v, nbr, agent, avail))
                        return cycle_edges
                if nbr not in visited:
                    rec_stack.append((v, (v, nbr, agent, avail)))
                    result = dfs(nbr)
                    if result is not None:
                        return result
                    rec_stack.pop()
            rec_stack.pop()
            return None

        for vertex in range(self.n):
            if vertex not in visited:
                cycle = dfs(vertex)
                if cycle is not None:
                    return cycle
        return None

    def execute_all_cycles(self):
        """
        現在の二重確率行列 self.P に対して、グラフを構築し、サイクルを探索しては
        そのサイクル内で交換可能な最小の確率 epsilon だけ交換を実施する。
        複数のサイクルが存在する場合、すべてのサイクルで交換が行われるまで反復する。
        交換が行われたサイクルと各サイクルでの epsilon を記録し、最終的な更新後行列 Q と
        サイクル情報のリストを返す。
        """
        Q = self.P.clone()
        cycles_exchanges = []
        while True:
            graph = self.build_graph(Q)
            cycle = self.find_cycle(graph)
            if cycle is None:
                break
            # サイクル内の各エッジの利用可能な確率の最小値を epsilon とする
            epsilons = [edge[3] for edge in cycle]
            epsilon = min(epsilons)
            # サイクル内の各エッジについて交換を実施
            for (a, b, agent, avail) in cycle:
                Q[agent, b] -= epsilon
                Q[agent, a] += epsilon
            cycles_exchanges.append((cycle, epsilon))
        return Q, cycles_exchanges

if __name__ == "__main__":
    # 例：2つの2人サイクルが存在するケース（4エージェント・4オブジェクト）
    P = torch.tensor([
        [0.7, 0.3, 0.0, 0.0],  # Agent0
        [0.3, 0.7, 0.0, 0.0],  # Agent1
        [0.0, 0.0, 0.6, 0.4],  # Agent2
        [0.0, 0.0, 0.4, 0.6]   # Agent3
    ])
    
    # 各行は {1, 0.75, 0.5, 0.25} の順列
    preferences = torch.tensor([
        [1.00, 0.75, 0.50, 0.25],
        [0.75, 1.00, 0.50, 0.25],
        [0.50, 0.25, 1.00, 0.75],
        [0.25, 0.50, 0.75, 1.00]
    ])
    
    adjuster = RandomAssignmentAdjuster(P, preferences)
    
    print("元の二重確率行列 P:")
    print(P)
    print("\n選好行列 (各行は [1, 0.75, 0.5, 0.25] の順列):")
    print(preferences)
    
    Q, cycles_exchanges = adjuster.execute_all_cycles()
    
    if cycles_exchanges:
        print("\n各サイクルでの交換内容:")
        for idx, (cycle, epsilon) in enumerate(cycles_exchanges):
            print(f"【サイクル {idx+1}】")
            for edge in cycle:
                print("  ", edge)
            print("  サイクル内で交換された確率 (epsilon):", epsilon)
    else:
        print("\nサイクルは検出されませんでした。")
    
    print("\n最終的な更新後の二重確率行列 Q:")
    print(Q)


元の二重確率行列 P:
tensor([[0.7000, 0.3000, 0.0000, 0.0000],
        [0.3000, 0.7000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.6000, 0.4000],
        [0.0000, 0.0000, 0.4000, 0.6000]])

選好行列 (各行は [1, 0.75, 0.5, 0.25] の順列):
tensor([[1.0000, 0.7500, 0.5000, 0.2500],
        [0.7500, 1.0000, 0.5000, 0.2500],
        [0.5000, 0.2500, 1.0000, 0.7500],
        [0.2500, 0.5000, 0.7500, 1.0000]])

各サイクルでの交換内容:
【サイクル 1】
   (0, 1, 0, 0.30000001192092896)
   (1, 0, 1, 0.30000001192092896)
  サイクル内で交換された確率 (epsilon): 0.30000001192092896
【サイクル 2】
   (2, 3, 2, 0.4000000059604645)
   (3, 2, 3, 0.4000000059604645)
  サイクル内で交換された確率 (epsilon): 0.4000000059604645

最終的な更新後の二重確率行列 Q:
tensor([[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]])


各サイクル内の和

In [ ]:
import torch

class RandomAssignmentAdjuster:
    def __init__(self, P, preferences):
        """
        P: n×n の二重確率行列 (torch.Tensor)
        preferences: n×n の選好行列 (torch.Tensor)
                     各行 i はエージェント i の選好を表し、値が大きいほど好む
        """
        self.P = P.clone()
        self.preferences = preferences
        self.n = P.shape[0]

    def build_graph(self, Q):
        """
        Q: 現在の二重確率行列 (torch.Tensor)
        オブジェクトを頂点とするグラフを構築する。
        エッジ (a -> b) は、あるエージェント i が a を b より好む（preferences[i, a] > preferences[i, b]）
        かつ Q[i, b] > 0 である場合に追加する。
        エッジには (b, i, Q[i, b]) の情報を記録する。
        """
        graph = {a: [] for a in range(self.n)}
        for a in range(self.n):
            for b in range(self.n):
                if a == b:
                    continue
                for i in range(self.n):
                    if self.preferences[i, a] > self.preferences[i, b] and Q[i, b] > 0:
                        graph[a].append((b, i, Q[i, b].item()))
                        # 同じ (a, b) ペアについて、最初に条件を満たしたエージェントを witness とする
                        break
        return graph

    def find_cycle(self, graph):
        """
        DFS を用いてグラフ中のサイクル（閉路）を探索する。
        サイクルが見つかった場合は、サイクルを構成するエッジのリストを返す。
        各エッジは (from_object, to_object, witness_agent, available_probability) のタプル。
        サイクルがなければ None を返す。
        """
        visited = set()
        rec_stack = []  # 各要素は (vertex, edge_info)。最初の頂点は edge_info=None

        def dfs(v):
            visited.add(v)
            rec_stack.append((v, None))
            for (nbr, agent, avail) in graph[v]:
                for idx, (node, _) in enumerate(rec_stack):
                    if node == nbr:
                        cycle_edges = []
                        # rec_stack[idx+1:] に記録されているエッジ情報がサイクル内のエッジ
                        for j in range(idx + 1, len(rec_stack)):
                            edge = rec_stack[j][1]
                            if edge is not None:
                                cycle_edges.append(edge)
                        cycle_edges.append((v, nbr, agent, avail))
                        return cycle_edges
                if nbr not in visited:
                    rec_stack.append((v, (v, nbr, agent, avail)))
                    result = dfs(nbr)
                    if result is not None:
                        return result
                    rec_stack.pop()
            rec_stack.pop()
            return None

        for vertex in range(self.n):
            if vertex not in visited:
                cycle = dfs(vertex)
                if cycle is not None:
                    return cycle
        return None

    def execute_all_cycles(self):
        """
        現在の二重確率行列 self.P に対して、グラフを構築し、サイクルを探索しては
        そのサイクル内で交換可能な最小の確率 epsilon だけ交換を実施する。
        複数のサイクルが存在する場合、すべてのサイクルで交換が行われるまで反復する。
        各サイクルで、サイクル内のエッジ数に epsilon を掛けた「交換された確率の和」も計算する。
        最終的な更新後行列 Q と、各サイクルのサイクル情報、epsilon、交換和のタプルのリストを返す。
        """
        Q = self.P.clone()
        cycles_exchanges = []
        while True:
            graph = self.build_graph(Q)
            cycle = self.find_cycle(graph)
            if cycle is None:
                break
            # サイクル内の各エッジの利用可能な確率の最小値を epsilon とする
            epsilons = [edge[3] for edge in cycle]
            epsilon = min(epsilons)
            # サイクル内で交換された確率の和は、サイクル内のエッジ数 * epsilon
            sum_exchanged = len(cycle) * epsilon
            # サイクル内の各エッジについて交換を実施
            for (a, b, agent, avail) in cycle:
                Q[agent, b] -= epsilon
                Q[agent, a] += epsilon
            cycles_exchanges.append((cycle, epsilon, sum_exchanged))
        return Q, cycles_exchanges

if __name__ == "__main__":
    # 例：2つの2人サイクルが存在するケース（4エージェント・4オブジェクト）
    P = torch.tensor([
        [0.7, 0.3, 0.0, 0.0],  # Agent0
        [0.3, 0.7, 0.0, 0.0],  # Agent1
        [0.0, 0.0, 0.6, 0.4],  # Agent2
        [0.0, 0.0, 0.4, 0.6]   # Agent3
    ])
    
    # 各行は {1, 0.75, 0.5, 0.25} の順列
    preferences = torch.tensor([
        [1.00, 0.75, 0.50, 0.25],
        [0.75, 1.00, 0.50, 0.25],
        [0.50, 0.25, 1.00, 0.75],
        [0.25, 0.50, 0.75, 1.00]
    ])
    
    adjuster = RandomAssignmentAdjuster(P, preferences)
    
    print("元の二重確率行列 P:")
    print(P)
    print("\n選好行列 (各行は [1, 0.75, 0.5, 0.25] の順列):")
    print(preferences)
    
    Q, cycles_exchanges = adjuster.execute_all_cycles()
    
    if cycles_exchanges:
        print("\n各サイクルでの交換内容:")
        for idx, (cycle, epsilon, sum_exchanged) in enumerate(cycles_exchanges):
            print(f"【サイクル {idx+1}】")
            for edge in cycle:
                print("  ", edge)
            print("  サイクル内で交換された確率 (epsilon):", epsilon)
            print("  サイクル内の交換確率の和:", sum_exchanged)
    else:
        print("\nサイクルは検出されませんでした。")
    
    print("\n最終的な更新後の二重確率行列 Q:")
    print(Q)


元の二重確率行列 P:
tensor([[0.7000, 0.3000, 0.0000, 0.0000],
        [0.3000, 0.7000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.6000, 0.4000],
        [0.0000, 0.0000, 0.4000, 0.6000]])

選好行列 (各行は [1, 0.75, 0.5, 0.25] の順列):
tensor([[1.0000, 0.7500, 0.5000, 0.2500],
        [0.7500, 1.0000, 0.5000, 0.2500],
        [0.5000, 0.2500, 1.0000, 0.7500],
        [0.2500, 0.5000, 0.7500, 1.0000]])

各サイクルでの交換内容:
【サイクル 1】
   (0, 1, 0, 0.30000001192092896)
   (1, 0, 1, 0.30000001192092896)
  サイクル内で交換された確率 (epsilon): 0.30000001192092896
  サイクル内の交換確率の和: 0.6000000238418579
【サイクル 2】
   (2, 3, 2, 0.4000000059604645)
   (3, 2, 3, 0.4000000059604645)
  サイクル内で交換された確率 (epsilon): 0.4000000059604645
  サイクル内の交換確率の和: 0.800000011920929

最終的な更新後の二重確率行列 Q:
tensor([[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]])


サイクル同士の和を考える

In [12]:
class RandomAssignmentAdjuster:
    def __init__(self, P, preferences):
        """
        P: n×n の二重確率行列 (torch.Tensor)
        preferences: n×n の選好行列 (torch.Tensor)
                     各行 i はエージェント i の選好を表し、値が大きいほど好む
        """
        self.P = P.clone()
        self.preferences = preferences
        self.n = P.shape[0]

    def build_graph(self, Q):
        """
        Q: 現在の二重確率行列 (torch.Tensor)
        オブジェクトを頂点とするグラフを構築する。
        エッジ (a -> b) は、あるエージェント i が a を b より好む（preferences[i, a] > preferences[i, b]）
        かつ Q[i, b] > 0 である場合に追加する。
        エッジには (b, i, Q[i, b]) の情報を記録する。
        """
        graph = {a: [] for a in range(self.n)}
        for a in range(self.n):
            for b in range(self.n):
                if a == b:
                    continue
                for i in range(self.n):
                    if self.preferences[i, a] > self.preferences[i, b] and Q[i, b] > 0:
                        graph[a].append((b, i, Q[i, b].item()))
                        # 同じ (a, b) ペアについて、最初に条件を満たしたエージェントを witness とする
                        break
        return graph

    def find_cycle(self, graph):
        """
        DFS を用いてグラフ中のサイクル（閉路）を探索する。
        サイクルが見つかった場合は、サイクルを構成するエッジのリストを返す。
        各エッジは (from_object, to_object, witness_agent, available_probability) のタプル。
        サイクルがなければ None を返す。
        """
        visited = set()
        rec_stack = []  # 各要素は (vertex, edge_info)。最初の頂点は edge_info=None

        def dfs(v):
            visited.add(v)
            rec_stack.append((v, None))
            for (nbr, agent, avail) in graph[v]:
                for idx, (node, _) in enumerate(rec_stack):
                    if node == nbr:
                        cycle_edges = []
                        # rec_stack[idx+1:] に記録されているエッジ情報がサイクル内のエッジ
                        for j in range(idx + 1, len(rec_stack)):
                            edge = rec_stack[j][1]
                            if edge is not None:
                                cycle_edges.append(edge)
                        cycle_edges.append((v, nbr, agent, avail))
                        return cycle_edges
                if nbr not in visited:
                    rec_stack.append((v, (v, nbr, agent, avail)))
                    result = dfs(nbr)
                    if result is not None:
                        return result
                    rec_stack.pop()
            rec_stack.pop()
            return None

        for vertex in range(self.n):
            if vertex not in visited:
                cycle = dfs(vertex)
                if cycle is not None:
                    return cycle
        return None

    def execute_all_cycles(self):
        """
        現在の二重確率行列 self.P に対して、グラフを構築し、サイクルを探索しては
        そのサイクル内で交換可能な最小の確率 epsilon だけ交換を実施する。
        複数のサイクルが存在する場合、すべてのサイクルで交換が行われるまで反復する。
        交換が行われたサイクルと各サイクルでの epsilon を記録し、最終的な更新後行列 Q と
        サイクル情報のリストを返す。
        """
        Q = self.P.clone()
        cycles_exchanges = []
        sum_epsilon = 0.0
        while True:
            graph = self.build_graph(Q)
            cycle = self.find_cycle(graph)
            if cycle is None:
                break
            # サイクル内の各エッジの利用可能な確率の最小値を epsilon とする
            epsilons = [edge[3] for edge in cycle]
            epsilon = min(epsilons)
            sum_epsilon += epsilon
            # サイクル内の各エッジについて交換を実施
            for (a, b, agent, avail) in cycle:
                Q[agent, b] -= epsilon
                Q[agent, a] += epsilon
            cycles_exchanges.append((cycle, epsilon))
        return Q, cycles_exchanges, sum_epsilon

上手くいってそう

In [13]:
# 例：2つの2人サイクルが存在するケース（4エージェント・4オブジェクト）
P = torch.tensor([
    [0.7, 0.3, 0.0, 0.0],  # Agent0
    [0.3, 0.7, 0.0, 0.0],  # Agent1
    [0.0, 0.0, 0.6, 0.4],  # Agent2
    [0.0, 0.0, 0.4, 0.6]   # Agent3
    ])
    
# 各行は {1, 0.75, 0.5, 0.25} の順列
preferences = torch.tensor([
    [1.00, 0.75, 0.50, 0.25],
    [0.75, 1.00, 0.50, 0.25],
    [0.50, 0.25, 1.00, 0.75],
    [0.25, 0.50, 0.75, 1.00]
    ])
    
adjuster = RandomAssignmentAdjuster(P, preferences)
    
print("元の二重確率行列 P:")
print(P)
print("\n選好行列 (各行は [1, 0.75, 0.5, 0.25] の順列):")
print(preferences)
    
Q, cycles_exchanges, sum_epsilon = adjuster.execute_all_cycles()
    
if cycles_exchanges:
    print("\n各サイクルでの交換内容:")
    for idx, (cycle, epsilon) in enumerate(cycles_exchanges):
        print(f"【サイクル {idx+1}】")
        for edge in cycle:
            print("  ", edge)
        print("  サイクル内で交換された確率 (epsilon):", epsilon)
    print("\n交換された確率の和:", sum_epsilon)
else:
     print("\nサイクルは検出されませんでした。")

print("\n最終的な更新後の二重確率行列 Q:")
print(Q)


元の二重確率行列 P:
tensor([[0.7000, 0.3000, 0.0000, 0.0000],
        [0.3000, 0.7000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.6000, 0.4000],
        [0.0000, 0.0000, 0.4000, 0.6000]])

選好行列 (各行は [1, 0.75, 0.5, 0.25] の順列):
tensor([[1.0000, 0.7500, 0.5000, 0.2500],
        [0.7500, 1.0000, 0.5000, 0.2500],
        [0.5000, 0.2500, 1.0000, 0.7500],
        [0.2500, 0.5000, 0.7500, 1.0000]])

各サイクルでの交換内容:
【サイクル 1】
   (0, 1, 0, 0.30000001192092896)
   (1, 0, 1, 0.30000001192092896)
  サイクル内で交換された確率 (epsilon): 0.30000001192092896
【サイクル 2】
   (2, 3, 2, 0.4000000059604645)
   (3, 2, 3, 0.4000000059604645)
  サイクル内で交換された確率 (epsilon): 0.4000000059604645

交換された確率の和: 0.7000000178813934

最終的な更新後の二重確率行列 Q:
tensor([[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]])


In [15]:
P = torch.tensor([
        [0.4, 0.3, 0.0, 0.3],
        [0.0, 0.4, 0.3, 0.3],
        [0.3, 0.3, 0.4, 0.0],
        [0.3, 0.0, 0.3, 0.4]
    ])

preferences = torch.tensor([
        [1.00, 0.75, 0.5,  0.25],   # Agent0
        [0.75, 1.00, 0.5,  0.25],   # Agent1
        [0.75, 0.5,  1.00, 0.25],   # Agent2
        [0.75, 0.5,  0.25, 1.00]     # Agent3
    ])

adjuster = RandomAssignmentAdjuster(P, preferences)
    
print("元の二重確率行列 P:")
print(P)
print("\n選好行列 (各行は [1, 0.75, 0.5, 0.25] の順列):")
print(preferences)
    
Q, cycles_exchanges, sum_epsilon = adjuster.execute_all_cycles()
    
if cycles_exchanges:
    print("\n各サイクルでの交換内容:")
    for idx, (cycle, epsilon) in enumerate(cycles_exchanges):
        print(f"【サイクル {idx+1}】")
        for edge in cycle:
            print("  ", edge)
        print("  サイクル内で交換された確率 (epsilon):", epsilon)
    print("\n交換された確率の和:", sum_epsilon)
else:
     print("\nサイクルは検出されませんでした。")

print("\n最終的な更新後の二重確率行列 Q:")
print(Q)

元の二重確率行列 P:
tensor([[0.4000, 0.3000, 0.0000, 0.3000],
        [0.0000, 0.4000, 0.3000, 0.3000],
        [0.3000, 0.3000, 0.4000, 0.0000],
        [0.3000, 0.0000, 0.3000, 0.4000]])

選好行列 (各行は [1, 0.75, 0.5, 0.25] の順列):
tensor([[1.0000, 0.7500, 0.5000, 0.2500],
        [0.7500, 1.0000, 0.5000, 0.2500],
        [0.7500, 0.5000, 1.0000, 0.2500],
        [0.7500, 0.5000, 0.2500, 1.0000]])

各サイクルでの交換内容:
【サイクル 1】
   (0, 1, 0, 0.30000001192092896)
   (1, 2, 1, 0.30000001192092896)
   (2, 0, 2, 0.30000001192092896)
  サイクル内で交換された確率 (epsilon): 0.30000001192092896
【サイクル 2】
   (1, 2, 3, 0.30000001192092896)
   (2, 1, 2, 0.30000001192092896)
  サイクル内で交換された確率 (epsilon): 0.30000001192092896
【サイクル 3】
   (0, 1, 3, 0.30000001192092896)
   (1, 3, 0, 0.30000001192092896)
   (3, 0, 3, 0.30000001192092896)
  サイクル内で交換された確率 (epsilon): 0.30000001192092896
【サイクル 4】
   (0, 1, 0, 0.30000001192092896)
   (1, 3, 1, 0.30000001192092896)
   (3, 0, 3, 0.30000001192092896)
  サイクル内で交換された確率 (epsilon): 0.30000001192092896


if文なくてもちゃんと出力されるか確認

In [ ]:
# 例：2つの2人サイクルが存在するケース（4エージェント・4オブジェクト）
P = torch.tensor([
    [1.0, 0.0, 0.0, 0.0],  # Agent0
    [0.0, 1.0, 0.0, 0.0],  # Agent1
    [0.0, 0.0, 1.0, 0.0],  # Agent2
    [0.0, 0.0, 0.0, 1.0]   # Agent3
    ])
    
# 各行は {1, 0.75, 0.5, 0.25} の順列
preferences = torch.tensor([
    [1.00, 0.75, 0.50, 0.25],
    [0.75, 1.00, 0.50, 0.25],
    [0.50, 0.25, 1.00, 0.75],
    [0.25, 0.50, 0.75, 1.00]
    ])
    
adjuster = RandomAssignmentAdjuster(P, preferences)
    
print("元の二重確率行列 P:")
print(P)
print("\n選好行列 (各行は [1, 0.75, 0.5, 0.25] の順列):")
print(preferences)
    
Q, cycles_exchanges, sum_epsilon = adjuster.execute_all_cycles()
    

print("\n各サイクルでの交換内容:")
for idx, (cycle, epsilon) in enumerate(cycles_exchanges):
    print(f"【サイクル {idx+1}】")
    for edge in cycle:
        print("  ", edge)
    print("  サイクル内で交換された確率 (epsilon):", epsilon)
print("\n交換された確率の和:", sum_epsilon)

print("\n最終的な更新後の二重確率行列 Q:")
print(Q)


元の二重確率行列 P:
tensor([[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]])

選好行列 (各行は [1, 0.75, 0.5, 0.25] の順列):
tensor([[1.0000, 0.7500, 0.5000, 0.2500],
        [0.7500, 1.0000, 0.5000, 0.2500],
        [0.5000, 0.2500, 1.0000, 0.7500],
        [0.2500, 0.5000, 0.7500, 1.0000]])

各サイクルでの交換内容:

交換された確率の和: 0.0

交換された確率の和: 0.0

最終的な更新後の二重確率行列 Q:
tensor([[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]])


In [ ]:
# 例：2つの2人サイクルが存在するケース（4エージェント・4オブジェクト）
P = torch.tensor([
    [0.7, 0.3, 0.0, 0.0],  # Agent0
    [0.3, 0.7, 0.0, 0.0],  # Agent1
    [0.0, 0.0, 0.6, 0.4],  # Agent2
    [0.0, 0.0, 0.4, 0.6]   # Agent3
    ])
    
# 各行は {1, 0.75, 0.5, 0.25} の順列
preferences = torch.tensor([
    [1.00, 0.75, 0.50, 0.25],
    [0.75, 1.00, 0.50, 0.25],
    [0.50, 0.25, 1.00, 0.75],
    [0.25, 0.50, 0.75, 1.00]
    ])

adjuster = RandomAssignmentAdjuster(P, preferences)
    
print("元の二重確率行列 P:")
print(P)
print("\n選好行列 (各行は [1, 0.75, 0.5, 0.25] の順列):")
print(preferences)
    
Q, cycles_exchanges, sum_epsilon = adjuster.execute_all_cycles()
    

print("\n各サイクルでの交換内容:")
for idx, (cycle, epsilon) in enumerate(cycles_exchanges):
    print(f"【サイクル {idx+1}】")
    for edge in cycle:
        print("  ", edge)
    print("  サイクル内で交換された確率 (epsilon):", epsilon)
print("\n交換された確率の和:", sum_epsilon)

print("\n最終的な更新後の二重確率行列 Q:")
print(Q)


元の二重確率行列 P:
tensor([[0.7000, 0.3000, 0.0000, 0.0000],
        [0.3000, 0.7000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.6000, 0.4000],
        [0.0000, 0.0000, 0.4000, 0.6000]])

選好行列 (各行は [1, 0.75, 0.5, 0.25] の順列):
tensor([[1.0000, 0.7500, 0.5000, 0.2500],
        [0.7500, 1.0000, 0.5000, 0.2500],
        [0.5000, 0.2500, 1.0000, 0.7500],
        [0.2500, 0.5000, 0.7500, 1.0000]])

各サイクルでの交換内容:
【サイクル 1】
   (0, 1, 0, 0.30000001192092896)
   (1, 0, 1, 0.30000001192092896)
  サイクル内で交換された確率 (epsilon): 0.30000001192092896
【サイクル 2】
   (2, 3, 2, 0.4000000059604645)
   (3, 2, 3, 0.4000000059604645)
  サイクル内で交換された確率 (epsilon): 0.4000000059604645

交換された確率の和: 0.7000000178813934

最終的な更新後の二重確率行列 Q:
tensor([[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]])


交換された確率の和のみ出力されるように

In [ ]:
class compute_ev:
    def __init__(self, P, preferences):
        """
        P: n×n の二重確率行列 (torch.Tensor)
        preferences: n×n の選好行列 (torch.Tensor)
                     各行 i はエージェント i の選好を表し、値が大きいほど好む
        """
        self.P = P.clone()
        self.preferences = preferences
        self.n = P.shape[0]

    def build_graph(self, Q):
        """
        Q: 現在の二重確率行列 (torch.Tensor)
        オブジェクトを頂点とするグラフを構築する。
        エッジ (a -> b) は、あるエージェント i が a を b より好む（preferences[i, a] > preferences[i, b]）
        かつ Q[i, b] > 0 である場合に追加する。
        エッジには (b, i, Q[i, b]) の情報を記録する。
        """
        graph = {a: [] for a in range(self.n)}
        for a in range(self.n):
            for b in range(self.n):
                if a == b:
                    continue
                for i in range(self.n):
                    if self.preferences[i, a] > self.preferences[i, b] and Q[i, b] > 0:
                        graph[a].append((b, i, Q[i, b].item()))
                        # 同じ (a, b) ペアについて、最初に条件を満たしたエージェントを witness とする
                        break
        return graph

    def find_cycle(self, graph):
        """
        DFS を用いてグラフ中のサイクル（閉路）を探索する。
        サイクルが見つかった場合は、サイクルを構成するエッジのリストを返す。
        各エッジは (from_object, to_object, witness_agent, available_probability) のタプル。
        サイクルがなければ None を返す。
        """
        visited = set()
        rec_stack = []  # 各要素は (vertex, edge_info)。最初の頂点は edge_info=None

        def dfs(v):
            visited.add(v)
            rec_stack.append((v, None))
            for (nbr, agent, avail) in graph[v]:
                for idx, (node, _) in enumerate(rec_stack):
                    if node == nbr:
                        cycle_edges = []
                        # rec_stack[idx+1:] に記録されているエッジ情報がサイクル内のエッジ
                        for j in range(idx + 1, len(rec_stack)):
                            edge = rec_stack[j][1]
                            if edge is not None:
                                cycle_edges.append(edge)
                        cycle_edges.append((v, nbr, agent, avail))
                        return cycle_edges
                if nbr not in visited:
                    rec_stack.append((v, (v, nbr, agent, avail)))
                    result = dfs(nbr)
                    if result is not None:
                        return result
                    rec_stack.pop()
            rec_stack.pop()
            return None

        for vertex in range(self.n):
            if vertex not in visited:
                cycle = dfs(vertex)
                if cycle is not None:
                    return cycle
        return None

    def execute_all_cycles(self):
        """
        現在の二重確率行列 self.P に対して、グラフを構築し、サイクルを探索しては
        そのサイクル内で交換可能な最小の確率 epsilon だけ交換を実施する。
        複数のサイクルが存在する場合、すべてのサイクルで交換が行われるまで反復する。
        交換が行われたサイクルと各サイクルでの epsilon を記録し、最終的な更新後行列 Q と
        サイクル情報のリストを返す。
        """
        Q = self.P.clone()
        cycles_exchanges = []
        ev = 0.0
        while True:
            graph = self.build_graph(Q)
            cycle = self.find_cycle(graph)
            if cycle is None:
                break
            # サイクル内の各エッジの利用可能な確率の最小値を epsilon とする
            epsilons = [edge[3] for edge in cycle]
            epsilon = min(epsilons)
            ev += epsilon
            # サイクル内の各エッジについて交換を実施
            for (a, b, agent, avail) in cycle:
                Q[agent, b] -= epsilon
                Q[agent, a] += epsilon
            cycles_exchanges.append((cycle, epsilon))
        return ev

In [ ]:
# 例：2つの2人サイクルが存在するケース（4エージェント・4オブジェクト）
P = torch.tensor([
    [0.7, 0.3, 0.0, 0.0],  # Agent0
    [0.3, 0.7, 0.0, 0.0],  # Agent1
    [0.0, 0.0, 0.6, 0.4],  # Agent2
    [0.0, 0.0, 0.4, 0.6]   # Agent3
    ])
    
# 各行は {1, 0.75, 0.5, 0.25} の順列
preferences = torch.tensor([
    [1.00, 0.75, 0.50, 0.25],
    [0.75, 1.00, 0.50, 0.25],
    [0.50, 0.25, 1.00, 0.75],
    [0.25, 0.50, 0.75, 1.00]
    ])

adjuster = compute_ev(P, preferences)

sum_epsilon = adjuster.execute_all_cycles()
    
print("\n交換された確率の和:", sum_epsilon)


交換された確率の和: 0.7000000178813934


Equal treatment of equals

In [ ]:
import torch

def violation_degree(P, preferences):
    """
    P: (n x n) の確率配分行列（各行はエージェントの割当確率ベクトル）
    preferences: (n x m) の選好行列（各行はエージェントの各財に対する評価）
    """
    n = preferences.shape[0]
    violation = 0.0
    equal_pair_count = 0
    
    # 各エージェントペアについて、選好が完全に一致していればその差を計算
    for i in range(n):
        for j in range(i + 1, n):
            # 全成分が等しければ同等とみなす
            if torch.all(preferences[i] == preferences[j]):
                diff = torch.sum(torch.abs(P[i] - P[j]))
                violation += diff
                equal_pair_count += 1
    return violation, equal_pair_count

In [ ]:
# 例: 選好行列 (4人×4財) 
preferences = torch.tensor([
    [1.00, 0.75, 0.50, 0.25],
    [1.00, 0.75, 0.50, 0.25],
    [0.75, 1.00, 0.50, 0.25],
    [0.75, 1.00, 0.25, 0.50]
])

# 例: 確率配分行列 (ここでは単純な例として4人とも均等割当)
P = torch.tensor([
    [0.25, 0.25, 0.25, 0.25],
    [0.25, 0.25, 0.25, 0.25],
    [0.30, 0.20, 0.30, 0.20],
    [0.20, 0.30, 0.20, 0.30]
])

violation, count = violation_degree(P, preferences)
print("Equal treatment of equals の違反度合い:", violation.item())
print("同一選好のエージェントペア数:", count)

Equal treatment of equals の違反度合い: 0.0
同一選好のエージェントペア数: 1


In [ ]:
# 選好行列の例 (4人×4財)
# エージェント 1 と 2 は同一の選好を持つ
preferences = torch.tensor([
    [1.00, 0.75, 0.50, 0.25],
    [1.00, 0.75, 0.50, 0.25],
    [0.75, 1.00, 0.50, 0.25],
    [0.75, 1.00, 0.25, 0.50]
])

# 確率配分行列の例
# エージェント 1 と 2 は同一選好ですが、割当が異なる
P = torch.tensor([
    [0.20, 0.30, 0.30, 0.20],  # エージェント 1
    [0.25, 0.25, 0.25, 0.25],  # エージェント 2 (同じ選好ながら異なる割当)
    [0.30, 0.30, 0.20, 0.20],  # エージェント 3
    [0.25, 0.25, 0.25, 0.25]   # エージェント 4
])

violation, count = violation_degree(P, preferences)
print("Equal treatment of equals の違反度合い:", violation)
print("同一選好のエージェントペア数:", count)

Equal treatment of equals の違反度合い: tensor(0.2000)
同一選好のエージェントペア数: 1


tensorではなくfloatになるように修正

In [ ]:
import torch

def violation_degree(P, preferences):
    """
    P: (n x n) の確率配分行列。各行はエージェントの割当確率ベクトルを表す。
    preferences: (n x m) の選好行列。各行はエージェントの各財に対する評価を表し、
                 数値が大きいほど好ましいと解釈される。
    """
    n = preferences.shape[0]
    violation = 0.0  # Pythonのfloatとして初期化
    equal_pair_count = 0
    
    # 全てのエージェントペア (i, j) について、選好が完全一致していれば検証
    for i in range(n):
        for j in range(i + 1, n):
            if torch.all(preferences[i] == preferences[j]):
                # L1ノルムを計算し、.item()で数値に変換
                diff = torch.sum(torch.abs(P[i] - P[j])).item()
                violation += diff
                equal_pair_count += 1
    return violation, equal_pair_count

In [ ]:
# 例: 選好行列 (4人×4財)
preferences = torch.tensor([
    [1.00, 0.75, 0.50, 0.25],
    [1.00, 0.75, 0.50, 0.25],
    [0.75, 1.00, 0.50, 0.25],
    [0.75, 1.00, 0.25, 0.50]
])

# 例: 確率配分行列
# エージェント 1 と 2 は同一の選好ですが、割当が異なるため違反度合いが発生
P = torch.tensor([
    [0.20, 0.30, 0.30, 0.20],  # エージェント 1
    [0.25, 0.25, 0.25, 0.25],  # エージェント 2 (同じ選好ながら異なる割当)
    [0.30, 0.30, 0.20, 0.20],  # エージェント 3
    [0.25, 0.25, 0.25, 0.25]   # エージェント 4
])

violation, count = violation_degree(P, preferences)
print("Equal treatment of equals の違反度合い:", violation)  # 単なる数値が出力される
print("同一選好のエージェントペア数:", count)

Equal treatment of equals の違反度合い: 0.20000001788139343
同一選好のエージェントペア数: 1


In [ ]:
# 選好行列の例 (4 人×4 財)
# グループ 1: エージェント 0 と 1 の選好は同一
# グループ 2: エージェント 2 と 3 の選好は同一（ただし、グループ 1 とは異なる）
preferences = torch.tensor([
    [1.00, 0.75, 0.50, 0.25],  # エージェント 0
    [1.00, 0.75, 0.50, 0.25],  # エージェント 1
    [0.75, 1.00, 0.50, 0.25],  # エージェント 2
    [0.75, 1.00, 0.50, 0.25]   # エージェント 3
])

# 確率配分行列の例 (4人×4財)
# 同一選好のグループ内で割当が異なるため、違反度合いが発生します。
P = torch.tensor([
    [0.20, 0.30, 0.30, 0.20],  # エージェント 0
    [0.25, 0.25, 0.25, 0.25],  # エージェント 1 (グループ 1内で異なる)
    [0.30, 0.30, 0.20, 0.20],  # エージェント 2
    [0.25, 0.25, 0.25, 0.25]   # エージェント 3 (グループ 2内で異なる)
])

violation, count = violation_degree(P, preferences)
print("Equal treatment of equals の違反度合い:", violation)
print("同一選好のエージェントペア数:", count)

Equal treatment of equals の違反度合い: 0.40000003576278687
同一選好のエージェントペア数: 2


In [ ]:
# 選好行列 (4人×4財)
# エージェント0,1,2は同一の選好 [1.00, 0.75, 0.50, 0.25] を持ち、
# エージェント3は異なる選好 [0.75, 1.00, 0.25, 0.50] を持つ
preferences = torch.tensor([
    [1.00, 0.75, 0.50, 0.25],  # エージェント0
    [1.00, 0.75, 0.50, 0.25],  # エージェント1
    [1.00, 0.75, 0.50, 0.25],  # エージェント2
    [0.75, 1.00, 0.25, 0.50]   # エージェント3 (異なる選好)
])

# 確率配分行列 (4×4)
# エージェント0,1,2は同一選好ですが、割当が異なるため違反度合いが発生
P = torch.tensor([
    [0.30, 0.30, 0.20, 0.20],  # エージェント0
    [0.25, 0.25, 0.25, 0.25],  # エージェント1
    [0.20, 0.30, 0.30, 0.20],  # エージェント2
    [0.25, 0.25, 0.25, 0.25]   # エージェント3
])

violation, count = violation_degree(P, preferences)
print("Equal treatment of equals の違反度合い:", violation)  # 期待される出力: 0.60
print("同一選好のエージェントペア数:", count)               # 期待される出力: 3

Equal treatment of equals の違反度合い: 0.6000000536441803
同一選好のエージェントペア数: 3


In [ ]:
# 4人全員が同一の選好 [1.0, 0.75, 0.5, 0.25] を持つ
preferences = torch.tensor([
    [1.0, 0.75, 0.5, 0.25],
    [1.0, 0.75, 0.5, 0.25],
    [1.0, 0.75, 0.5, 0.25],
    [1.0, 0.75, 0.5, 0.25]
])

# 確率配分行列 (4人×4オブジェクト)
P = torch.tensor([
    [0.4,  0.3,  0.2,  0.1],   # エージェント0
    [0.3,  0.3,  0.2,  0.2],   # エージェント1
    [0.25, 0.25, 0.25, 0.25],   # エージェント2
    [0.1,  0.2,  0.3,  0.4]     # エージェント3
])

violation = violation_degree(P, preferences)
print("Equal treatment of equals の違反度合い:", violation)  # 期待される出力: 2.6

Equal treatment of equals の違反度合い: 2.600000113248825


violationのみ出力

In [ ]:
import torch
import numpy as np

def violation_degree(P, preferences):
    """
    P: (n x n) の確率配分行列。各行はエージェントの割当確率ベクトルを表す。
    preferences: (n x m) の選好行列。各行はエージェントの各財に対する評価を表し、
                 数値が大きいほど好ましいと解釈される。
    """
    n = preferences.shape[0]
    violation = 0.0  # Pythonのfloatとして初期化
    
    # 全てのエージェントペア (i, j) について、選好が完全一致していれば検証
    for i in range(n):
        for j in range(i + 1, n):
            if torch.all(preferences[i] == preferences[j]):
                # L1ノルムを計算し、.item()で数値に変換
                diff = torch.sum(torch.abs(P[i] - P[j])).item()
                violation += diff
    return violation

In [ ]:
import torch

def violation_degree(P, preferences):
    """
    P: (n x n) の確率配分行列（各行はエージェントの割当確率ベクトル）
    preferences: (n x m) の選好行列（各行はエージェントの各財に対する評価）
    """
    n = preferences.shape[0]
    violation = 0.0
    equal_pair_count = 0
    
    # 各エージェントペアについて、選好が完全に一致していればその差を計算
    for i in range(n):
        for j in range(i + 1, n):
            # 全成分が等しければ同等とみなす
            if torch.all(preferences[i] == preferences[j]):
                diff = torch.sum(torch.abs(P[i] - P[j]))
                violation += diff
                equal_pair_count += 1
    return violation

In [ ]:
preferences = torch.tensor([[
    [1.0, 0.75, 0.5, 0.25],
    [1.0, 0.75, 0.5, 0.25],
    [1.0, 0.75, 0.5, 0.25],
    [1.0, 0.75, 0.5, 0.25]
],[
    [1.0, 0.75, 0.5, 0.25],
    [1.0, 0.75, 0.5, 0.25],
    [1.0, 0.75, 0.5, 0.25],
    [1.0, 0.75, 0.5, 0.25]
]])

# 確率配分行列 (4人×4オブジェクト)
P = torch.tensor([[
    [0.4,  0.3,  0.2,  0.1],   # エージェント0
    [0.3,  0.3,  0.2,  0.2],   # エージェント1
    [0.25, 0.25, 0.25, 0.25],   # エージェント2
    [0.1,  0.2,  0.3,  0.4]     # エージェント3
],[
    [0.4,  0.3,  0.2,  0.1],   # エージェント0
    [0.3,  0.3,  0.2,  0.2],   # エージェント1
    [0.25, 0.25, 0.25, 0.25],   # エージェント2
    [0.1,  0.2,  0.3,  0.4]     # エージェント3
]])

violation = violation_degree(P, preferences)
print("Equal treatment of equals の違反度合い:", violation)  # 期待される出力: 2.6

Equal treatment of equals の違反度合い: tensor(0.)


In [3]:
import torch
import numpy as np

class compute_ev:
    def __init__(self, P, preferences):
        """
        P: n×n の二重確率行列 (torch.Tensor)
        preferences: n×n の選好行列 (torch.Tensor)
                     各行 i はエージェント i の選好を表し、値が大きいほど好む
        """
        self.P = P.clone()
        self.preferences = preferences
        self.n = P.shape[1]

    def build_graph(self, Q):
        graph = {a: [] for a in range(self.n)}
        for i in range(self.n):
            sorted_preferences = torch.argsort(self.preferences[i], descending=True)
            for a in sorted_preferences:
                for b in sorted_preferences:
                    if a == b:
                        continue
                    if Q[i, b] > 0:
                        if a not in graph:
                            graph[a] = []
                        graph[a].append((b, i, Q[i, b].item()))
                        break
        return graph
    
    def find_cycle(self, graph):
        visited = set()
        rec_stack = []

        def dfs(v):
            visited.add(v)
            rec_stack.append((v, None))
            for (nbr, agent, avail) in graph[v]:
                if nbr in visited:
                    cycle_edges = [(v, nbr, agent, avail)]
                    for node, edge in reversed(rec_stack):
                        if node == nbr:
                            break
                        cycle_edges.append(edge)
                    return cycle_edges
                if nbr not in visited:
                    rec_stack.append((v, (v, nbr, agent, avail)))
                    result = dfs(nbr)
                    if result is not None:
                        return result
                    rec_stack.pop()
            rec_stack.pop()
            return None

        for vertex in range(self.n):
            if vertex not in visited:
                cycle = dfs(vertex)
                if cycle is not None:
                    return cycle
        return None
    
    def execute_all_cycles(self):
        Q = self.P.clone()
        cycles_exchanges = []
        violation = 0.0
        while True:
            graph = self.build_graph(Q)
            cycle = self.find_cycle(graph)
            if cycle is None:
                break
            epsilons = [edge[3] for edge in cycle]
            epsilon = min(epsilons)
            violation += epsilon
            for (a, b, agent, avail) in cycle:
                Q[agent, b] -= epsilon
                Q[agent, a] += epsilon
            cycles_exchanges.append((cycle, epsilon))
        return violation

    def execute_all_cycles_batch(self):
        """
        P と preferences の各バッチに対して execute_all_cycles を計算し、
        それらの結果を合わせて n*1 の行列にする。
        """
        batch_size = self.P.shape[0]
        results = torch.zeros((batch_size, 1), dtype=torch.float32)

        for b in range(batch_size):
            P_batch = self.P[b].view(self.n, self.n)
            preferences_batch = self.preferences[b].view(self.n, self.n)
            ev_instance = compute_ev(P_batch, preferences_batch)
            results[b, 0] = ev_instance.execute_all_cycles()

        return results

In [16]:
import torch
import numpy as np
from concurrent.futures import ThreadPoolExecutor

class compute_ev:
    def __init__(self, P, preferences):
        """
        P: n×n の二重確率行列 (torch.Tensor)
        preferences: n×n の選好行列 (torch.Tensor)
                     各行 i はエージェント i の選好を表し、値が大きいほど好む
        """
        self.P = P.clone()
        self.preferences = preferences
        self.n = P.shape[1]

    def build_graph(self, Q):
        """
        Q: 現在の二重確率行列 (torch.Tensor)
        オブジェクトを頂点とするグラフを構築する。
        エッジ (a -> b) は、あるエージェント i が a を b より好む（preferences[i, a] > preferences[i, b]）
        かつ Q[i, b] > 0 である場合に追加する。
        エッジには (b, i, Q[i, b]) の情報を記録する。
        """
        graph = {a: [] for a in range(self.n)}
        for a in range(self.n):
            for b in range(self.n):
                if a == b:
                    continue
                for i in range(self.n):
                    if self.preferences[i, a] > self.preferences[i, b] and Q[i, b] > 0:
                        graph[a].append((b, i, Q[i, b].item())) 
                        # 同じ (a, b) ペアについて、最初に条件を満たしたエージェントを witness とする
                        break
        return graph

    def find_cycle(self, graph):
        """
        DFS を用いてグラフ中のサイクル（閉路）を探索する。
        サイクルが見つかった場合は、サイクルを構成するエッジのリストを返す。
        各エッジは (from_object, to_object, witness_agent, available_probability) のタプル。
        サイクルがなければ None を返す。
        """
        visited = set()
        rec_stack = []  # 各要素は (vertex, edge_info)。最初の頂点は edge_info=None

        def dfs(v):
            visited.add(v)
            rec_stack.append((v, None))
            for (nbr, agent, avail) in graph[v]:
                for idx, (node, _) in enumerate(rec_stack):
                    if node == nbr:
                        cycle_edges = []
                        # rec_stack[idx+1:] に記録されているエッジ情報がサイクル内のエッジ
                        for j in range(idx + 1, len(rec_stack)):
                            edge = rec_stack[j][1]
                            if edge is not None:
                                cycle_edges.append(edge)
                        cycle_edges.append((v, nbr, agent, avail))
                        return cycle_edges
                if nbr not in visited:
                    rec_stack.append((v, (v, nbr, agent, avail)))
                    result = dfs(nbr)
                    if result is not None:
                        return result
                    rec_stack.pop()
            rec_stack.pop()
            return None

        for vertex in range(self.n):
            if vertex not in visited:
                cycle = dfs(vertex)
                if cycle is not None:
                    return cycle
        return None

    def execute_all_cycles(self):
        """
        現在の二重確率行列 self.P に対して、グラフを構築し、サイクルを探索しては
        そのサイクル内で交換可能な最小の確率 epsilon だけ交換を実施する。
        複数のサイクルが存在する場合、すべてのサイクルで交換が行われるまで反復する。
        交換が行われたサイクルと各サイクルでの epsilon を記録し、最終的な更新後行列 Q と
        サイクル情報のリストを返す。
        """
        Q = self.P.clone()
        cycles_exchanges = []
        violation = 0.0
        while True:
            graph = self.build_graph(Q)
            cycle = self.find_cycle(graph)
            if cycle is None:
                break
            # サイクル内の各エッジの利用可能な確率の最小値を epsilon とする
            epsilons = [edge[3] for edge in cycle]
            epsilon = min(epsilons)
            violation += epsilon
            # サイクル内の各エッジについて交換を実施
            for (a, b, agent, avail) in cycle:
                Q[agent, b] -= epsilon
                Q[agent, a] += epsilon
            cycles_exchanges.append((cycle, epsilon))
        return violation

    def execute_all_cycles_batch(self):
        """
        P と preferences の各バッチに対して execute_all_cycles を計算し、
        それらの結果を合わせて n*1 の行列にする。
        """
        batch_size = self.P.shape[0]
        results = torch.zeros((batch_size, 1), dtype=torch.float32)

        def process_batch(b):
            P_batch = self.P[b].view(self.n, self.n)
            preferences_batch = self.preferences[b].view(self.n, self.n)
            ev_instance = compute_ev(P_batch, preferences_batch)
            return ev_instance.execute_all_cycles()

        with ThreadPoolExecutor() as executor:
            results_list = list(executor.map(process_batch, range(batch_size)))

        for b in range(batch_size):
            results[b, 0] = results_list[b]


        return results

In [7]:
preferences = torch.tensor([[
    [1.0, 0.75, 0.5, 0.25],
    [1.0, 0.75, 0.5, 0.25],
    [1.0, 0.75, 0.5, 0.25],
    [1.0, 0.75, 0.5, 0.25]
],[
    [1.0, 0.75, 0.5, 0.25],
    [1.0, 0.75, 0.5, 0.25],
    [1.0, 0.75, 0.5, 0.25],
    [1.0, 0.75, 0.5, 0.25]
]])

# 確率配分行列 (4人×4オブジェクト)
P = torch.tensor([[
    [0.4,  0.3,  0.2,  0.1],   # エージェント0
    [0.3,  0.3,  0.2,  0.2],   # エージェント1
    [0.25, 0.25, 0.25, 0.25],   # エージェント2
    [0.1,  0.2,  0.3,  0.4]     # エージェント3
],[
    [0.4,  0.3,  0.2,  0.1],   # エージェント0
    [0.3,  0.3,  0.2,  0.2],   # エージェント1
    [0.25, 0.25, 0.25, 0.25],   # エージェント2
    [0.1,  0.2,  0.3,  0.4]     # エージェント3
]])

adjuster = compute_ev(P, preferences)
print(adjuster.execute_all_cycles_batch())

tensor([[0.],
        [0.]])


In [17]:
# 例：2つの2人サイクルが存在するケース（4エージェント・4オブジェクト）
P = torch.tensor([[
    [0.7, 0.3, 0.0, 0.0],  # Agent0
    [0.3, 0.7, 0.0, 0.0],  # Agent1
    [0.0, 0.0, 0.6, 0.4],  # Agent2
    [0.0, 0.0, 0.4, 0.6]   # Agent3
    ],[
    [0.7, 0.3, 0.0, 0.0],  # Agent0
    [0.3, 0.7, 0.0, 0.0],  # Agent1
    [0.0, 0.0, 0.6, 0.4],  # Agent2
    [0.0, 0.0, 0.4, 0.6]   # Agent3
    ]])
    
# 各行は {1, 0.75, 0.5, 0.25} の順列
preferences = torch.tensor([[
    [1.00, 0.75, 0.50, 0.25],
    [0.75, 1.00, 0.50, 0.25],
    [0.50, 0.25, 1.00, 0.75],
    [0.25, 0.50, 0.75, 1.00]
    ],[
    [1.00, 0.75, 0.50, 0.25],
    [0.75, 1.00, 0.50, 0.25],
    [0.50, 0.25, 1.00, 0.75],
    [0.25, 0.50, 0.75, 1.00]
    ]])

adjuster = compute_ev(P, preferences)
print(adjuster.execute_all_cycles_batch())

tensor([[0.7000],
        [0.7000]])


In [18]:
P = torch.tensor([[
        [0.4, 0.3, 0.0, 0.3],
        [0.0, 0.4, 0.3, 0.3],
        [0.3, 0.3, 0.4, 0.0],
        [0.3, 0.0, 0.3, 0.4]
    ],[
        [0.4, 0.3, 0.0, 0.3],
        [0.0, 0.4, 0.3, 0.3],
        [0.3, 0.3, 0.4, 0.0],
        [0.3, 0.0, 0.3, 0.4]
    ]])

preferences = torch.tensor([[
        [1.00, 0.75, 0.5,  0.25],   # Agent0
        [0.75, 1.00, 0.5,  0.25],   # Agent1
        [0.75, 0.5,  1.00, 0.25],   # Agent2
        [0.75, 0.5,  0.25, 1.00]     # Agent3
    ],[
        [1.00, 0.75, 0.5,  0.25],   # Agent0
        [0.75, 1.00, 0.5,  0.25],   # Agent1
        [0.75, 0.5,  1.00, 0.25],   # Agent2
        [0.75, 0.5,  0.25, 1.00]     # Agent3
    ]])

adjuster = compute_ev(P, preferences)
print(adjuster.execute_all_cycles_batch())

tensor([[1.2000],
        [1.2000]])


In [11]:
P = torch.tensor([[
        [0.4, 0.3, 0.0, 0.3],
        [0.0, 0.4, 0.3, 0.3],
        [0.3, 0.3, 0.4, 0.0],
        [0.3, 0.0, 0.3, 0.4]
    ]])

preferences = torch.tensor([[
        [1.00, 0.75, 0.5,  0.25],   # Agent0
        [0.75, 1.00, 0.5,  0.25],   # Agent1
        [0.75, 0.5,  1.00, 0.25],   # Agent2
        [0.75, 0.5,  0.25, 1.00]     # Agent3
    ]])

adjuster = compute_ev(P, preferences)
print(adjuster.execute_all_cycles_batch())

tensor([[1.2000]])
